In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 577 ms (started: 2021-08-17 13:19:24 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 ch12 #1 全部更新 Aug 17, 2021'

git push origin master

In [2]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.93 s (started: 2021-08-17 13:19:39 +08:00)


# 使用 TensorFlow 定制模型和训练

到目前为止，我们只使用了 TensorFlow 的高级 API tf.keras，但它已经让我们走得很远：我们构建了各种神经网络架构，包括回归和分类网络、Wide & Deep 网络以及自标准化网络，使用各种技术，例如批量标准化、辍学和学习率计划。事实上，你将遇到的 95% 的用例除了 tf.keras（和 tf.data；参见第 13 章）之外不需要任何其他东西。但是现在是时候深入研究 TensorFlow 并查看其较低级别的 Python API。当您需要额外的控制来编写自定义损失函数、自定义指标、层、模型、初始化程序、正则化程序、权重约束等时，这将非常有用。您甚至可能需要完全控制训练循环本身，例如对梯度应用特殊转换或约束（不仅仅是裁剪它们）或对网络的不同部分使用多个优化器。我们将在本章中介绍所有这些情况，我们还将研究如何使用 TensorFlow 的自动图形生成功能来提升您的自定义模型和训练算法。但首先，让我们快速浏览一下 TensorFlow。

## TensorFlow 快速浏览

如您所知，TensorFlow 是一个强大的数值计算库，特别适合大规模机器学习并进行了微调（但您可以将它用于需要大量计算的其他任何事情）。 它由 Google Brain 团队开发，为 Google 的许多大型服务提供支持，例如 Google Cloud Speech、Google Photos 和 Google Search。 它于 2015 年 11 月开源，现在是最受欢迎的深度学习库（就论文引用、公司采用率、GitHub 上的明星等而言）。 无数项目将 TensorFlow 用于各种机器学习任务，例如图像分类、自然语言处理、推荐系统和时间序列预测。

那么 TensorFlow 提供了什么？ 这是一个总结：
* 其核心与NumPy 非常相似，但具有GPU 支持。
* 支持分布式计算（跨多个设备和服务器）。
* 它包括一种即时 (JIT) 编译器，允许它针对速度和内存使用情况优化计算。 它的工作原理是从 Python 函数中提取计算图，然后对其进行优化（例如，通过修剪未使用的节点），最后有效地运行它（例如，通过自动并行运行独立操作）。
* 计算图可以导出为可移植格式，因此您可以在一种环境中训练 TensorFlow 模型（例如，在 Linux 上使用 Python）并在另一种环境中运行它（例如，在 Android 设备上使用 Java）。
* 它实现了 autodiff（参见第 10 章和附录 D）并提供了一些优秀的优化器，例如 RMSProp 和 Nadam（参见第 11 章），因此您可以轻松地最小化各种损失函数。